In [22]:
# Importar bibiotecas
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
###
from src import config
import time
from datetime import datetime
from tqdm import tqdm
from src import dw_utils as dw
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [2]:
import platform

In [3]:
platform.system()

'Windows'

In [28]:
### atualzar lista de links no sitemap
BASE_URL = 'https://www.gruposhopmix.com/'
## obter lista de urls a partir do index

content = requests.get(config.URL_INDEX).text

### processar o html obtido 
### extracts all loc tags from content
soup = bs(content, 'html.parser')
all_locs = soup.find_all('loc')
urls = pd.DataFrame(columns = ["links_catalogo"], data=all_locs)

### remove keywords from urls
keywords = ['pagina','sao-paulo', 'todos-os-produtos', 'os-mais-vendidos', 'novidades','black-dos-campeoes']
urls = urls[~urls["links_catalogo"].str.contains('|'.join(keywords))]
## drop lines that is equal to the base url
urls = urls.loc[urls["links_catalogo"] != BASE_URL]
### obter dataset final com urls
urls.reset_index(inplace=True, drop=True)

c:\Users\bcesar\miniconda3\envs\scrape_shopmix\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [27]:

def extract_data_from_html(soup,url):
    
    

    nome = soup.find('meta', property='og:title')['content']

    codigo = soup.find('meta', attrs={'name': 'twitter:data1'})['content']
    try:
        preco = soup.find('strong', attrs = 'preco-promocional cor-principal titulo')['data-sell-price']
    except TypeError:
        preco = "N/A"
    disponibilidade = soup.find('meta', attrs={'name': 'twitter:data2'})['content']
    descricao = soup.find('meta', attrs={'name': 'description'})['content']
    try:
        disponibilidade_entrega = soup.find('span', class_='disponibilidade disp-entrega').find('b').text
    except AttributeError:
        disponibilidade_entrega = 'Não disponível'
    try:
        estoque = soup.find('b', class_='qtde_estoque').text
    except AttributeError:
        estoque = 'Não disponível'
    imagens = [img['src'] for img in soup.find_all('img') if img.get('src') != None]
    imagens = [imagem for imagem in imagens if 'produto' in imagem]
    
    return {
        'nome': nome,
        'código': codigo,
        'preço': preco,
        'descrição': descricao,
        'disponibilidade': disponibilidade,
        'disponibilidade_entrega': disponibilidade_entrega,
        'estoque': estoque,
        'imagens': imagens
    }

In [31]:
# obter soup de cada url de cada produto


df_produtos_consolidado = pd.DataFrame()
counter = 0
for link in tqdm(urls["links_catalogo"], desc='Obtendo dados dos produtos', unit='produto'):

    ## baixar pagina do produto
    prod_content= requests.get(link).text
    
    ### converter página em soup
    prod_content_soup = bs(prod_content, 'html.parser')
    ### extrair dicionário com os dados chave do produto ## converter dicionário em dataframe
    try:
        dict_produto = extract_data_from_html(prod_content_soup,link)
    except Exception as e:
        print(e, "___>" ,'Erro ao obter dados do produto: --->',link)
        continue
    ## converter a lita de imagens em str
    dict_produto["imagens"] = str(dict_produto["imagens"])

    ##criar um dataframe do produto
    df_produto = pd.DataFrame(dict_produto, index=[0])
    ###inserir o link do produto
    df_produto["link"] = link
    
    ### inseirr a data de extração
    df_produto["data_extracao"] = datetime.now()
    ## adicionar ao dataframe consolidado
    df_produtos_consolidado = pd.concat([df_produtos_consolidado, df_produto], ignore_index=True)
    ##colocar uma pausa de 1 segundo a cada 50 produtos
    counter += 1
    time.sleep(5)
    if counter  == 30:
        time.sleep(57)
        # break
        counter = 0
    

# salvar dataframe consolidado em csv

#df_produtos_consolidado.to_csv('data/produtos_consolidado.csv', index=False, sep='|')

### salvar dataframe consolidado no banco de dados



Obtendo dados dos produtos:   0%|          | 0/15 [00:00<?, ?produto/s]

Obtendo dados dos produtos: 100%|██████████| 15/15 [01:26<00:00,  5.77s/produto]


In [ ]:
### carregar no banco


if platform.system() == 'Windows':
    p_host = 'XXX'
    p_port = 5432
    db = 'XXX'
    ssh = True
    ssh_user = 'XXX'
    ssh_host = 'XXX'
    psql_user = 'XXX'


    psql_pass = 'XXX'
    ssh_pkey = r"XXXX"

    pgres = dw.Postgresql_connect(pgres_host=p_host, pgres_port=p_port, db=db, ssh=ssh, ssh_user=ssh_user, ssh_host=ssh_host, ssh_pkey=ssh_pkey, psql_user=psql_user
                                , psql_pass=psql_pass)
    #initiates a connection to the PostgreSQL database. In this instance we use ssh and must specify our ssh credentials.

    #You'll need to define psql_user and psql_pass using input() and getpass() to temporarily store your credentials.
    #Alternatively, best practice you may be to store your credentials as environment variables.
    # psql_user = input("Please enter your database username:")


elif platform.system() == 'Linux':
    p_host = 'XXX'
    p_port = 5432
    db = 'XXX'
    ssh = False
    ssh_user = 'XXXX'
    ssh_host = 'XXX'
    psql_user = 'XXXXs'
    
    psql_pass = 'XXX'
    ssh_pkey = r"XXXX"
    
    pgres = dw.Postgresql_connect(pgres_host=p_host, pgres_port=p_port, db=db, ssh=ssh, ssh_user=ssh_user, ssh_host=ssh_host, ssh_pkey=ssh_pkey, psql_user=psql_user
                                , psql_pass=psql_pass)

Server connected via SSH || Local Port: 57623...


In [8]:
#df_produtos_consolidado = pd.read_csv('data/produtos_consolidado.csv', sep='|')

In [9]:
### criar conexão com o banco de dados
## atualizar tabela

pgres.replace_table(df_produtos_consolidado, 'catalogo_shopmix')

Database [postgres] session created...
<> Table Replaced <>


2023-11-17 18:27:05,777| ERROR   | Socket exception: Foi forçado o cancelamento de uma conexão existente pelo host remoto (10054)


In [90]:


# sql_statement = """
# SELECT *
# FROM catalogo_shopmix
# ;
# """
# query_df = pgres.query( query=sql_statement)
# query_df
# #returns the results of an sql statement as a pandas dataframe. 
# #This example returns the column names and data types of table 'ey_test_table'.

Database [postgres] session created...
<> Query Sucessful <>


,nome,código,preço,descrição,disponibilidade,disponibilidade_entrega,estoque,imagens,link,data_extracao
0,Porta Joias Portátil Viagem Estojo Maleta Caix...,STO1761,18.9,Porta Joias Portátil Viagem Estojo Maleta Caix...,Disponível,Imediata,1002,['https://cdn.awsli.com.br/64x50/1080/1080773/...,https://www.gruposhopmix.com/porta-joias-porta...,2023-07-13 22:26:57.815579
1,Luva Protetora Anti Corte para Cozinha Tecido ...,STO707,21.9,Luva Anti-CorteAntibacterianaCozinha Mecânico ...,Disponível,Imediata,1017,['https://cdn.awsli.com.br/64x50/1080/1080773/...,https://www.gruposhopmix.com/luva-protetora-an...,2023-07-13 22:27:04.303295
2,Shockpad Anti-vibração Para Máquina De Lavar E...,STO874,12.9,Shockpad Anti-vibração Para Máquina De Lavar E...,Disponível,Imediata,1001,['https://cdn.awsli.com.br/64x50/1080/1080773/...,https://www.gruposhopmix.com/produto/shockpad-...,2023-07-13 22:27:10.694891
3,Caneca Térmica Com Tampa 380 ml Mixer A Pilha,STO1399,64.9,Caneca Mixer Misturadora Magnética Copo Térmic...,Disponível,Imediata,1007,['https://cdn.awsli.com.br/64x50/1080/1080773/...,https://www.gruposhopmix.com/caneca-termica-co...,2023-07-13 22:27:17.265940
4,"Balança Digital Pesar Ouro 0,1 Gr Até 500 Gramas",STO252,26.9,"Balança Digital Pesar Ouro 0,1 Gr Até 500 Gram...",Disponível,Imediata,1001,['https://cdn.awsli.com.br/64x50/1080/1080773/...,https://www.gruposhopmix.com/produto/balanca-d...,2023-07-13 22:29:26.074834
...,...,...,...,...,...,...,...,...,...,...
1506,Estante Prateleira 30cm 2 Andares Suporte Orga...,STO1570,29.9,Estante Prateleira Para Banheiro Sobre Vaso Su...,Indisponível,Não disponível,Não disponível,['https://cdn.awsli.com.br/64x50/1080/1080773/...,https://www.gruposhopmix.com/produto/estante-p...,2023-07-14 01:23:49.874893
1507,Festão Verde Claro 2m X 12cm Grosso Flexível D...,STO1530,NaN,Festão Verde Claro 2m X 12cm Grosso Flexível D...,Indisponível,Não disponível,Não disponível,['https://cdn.awsli.com.br/64x50/1080/1080773/...,https://www.gruposhopmix.com/produto/festao-ve...,2023-07-14 01:23:56.877331
1508,Guirlanda De Natal Enfeite Papai Noel Com Beng...,STO1523PB,69.9,Guirlanda De Natal Enfeite Papai Noel Com Beng...,Indisponível,Não disponível,Não disponível,['https://cdn.awsli.com.br/64x50/1080/1080773/...,https://www.gruposhopmix.com/produto/guirlanda...,2023-07-14 01:24:03.379566
1509,Kit Copa Do Mundo Seleção Brasil Individual Ch...,STO1394,29.9,Kit Copa Do Mundo Torcedor Seleção Brasil Indi...,Indisponível,Não disponível,Não disponível,['https://cdn.awsli.com.br/64x50/1080/1080773/...,https://www.gruposhopmix.com/kit-copa-do-mundo...,2023-07-14 01:24:09.888665
